In [10]:
import PyPlot

using AircraftDesignTools
adt = AircraftDesignTools
gt = adt.gt

data_path = "data/"
airfoil_path = joinpath(data_path, "airfoil/")

save_path = "temps/trainer00/"
gt.create_path(save_path, true)

# Visionary Concept

## Component Database

In [2]:
# Motor #1
motor1000kv_shape = ShapeCyl(                       # Cylindrical shape
                        36/2 / 1000,                # (m) radius
                        53 / 1000,                  # (m) height
                        "m"                         # Length units
                       )
motor1000kv_obj = object_from_mass(                 # Calc density from mass&shape
                        motor1000kv_shape,          # Shape
                        52 / 1000;                  # (kg) mass
                        massunits="kg"              # Mass units
                       )
motor1000kv = Component(                            # Component definition
                        "Motor 1000kv",             # Name
                        motor1000kv_obj;            # Object
                        id="motor1000kv",           # Identifier
                        description="Turnigy D2830-11 1000kv Brushless Motor",
                        comments="1000kv, requires 30A ESC, "*
                                    "suggested prop 8x4 (4S) ~ 10x7 (2S) ",
                        vendor="https://hobbyking.com/en_us/d2830-11-1000kv-brushless-motor.html",
                        cost=11                     # $/unit
                       )

# Motor #2
motor3040kv_shape = ShapeCyl(                       # Cylindrical shape
                        36/2 / 1000,                # (m) radius
                        53 / 1000,                  # (m) height
                        "m"                         # Length units
                       )
motor3040kv_obj = object_from_mass(                 # Calc density from mass&shape
                        motor3040kv_shape,          # Shape
                        178 / 1000;                 # (kg) mass
                        massunits="kg"              # Mass units
                       )
motor3040kv = Component(                            # Component definition
                        "Motor 3040kv",             # Name
                        motor3040kv_obj;            # Object
                        id="motor3040kv",           # Identifier
                        description="Turnigy TrackStar 13.5T Sensored Brushless Motor 3040KV",
                        comments="3040kv, max 36A ESC, ",
                        vendor="https://hobbyking.com/en_us/turnigy-trackstar-13-5t-sensored-brushless-motor-3040kv-roar-approved.html",
                        cost=35.26                  # $/unit
                       )

# Motor #3
motor192kv_shape = ShapePoint()
motor192kv_obj = object_from_mass(                  # Calc density from mass&shape
                        motor192kv_shape,           # Shape
                        858 / 1000;                 # (kg) mass
                        massunits="kg",             # Mass units
                        objecttype=ObjectPoint      # Defines it as a point mass
                       )
motor192kv = Component(                             # Component definition
                        "Motor 192kv",              # Name
                        motor192kv_obj;             # Object
                        id="motor192kv",            # Identifier
                        description="Turnigy Aerodrive SK3 - 6374-192KV Brushless Outrunner Motor",
                        comments="192kv, max 80A ESC, ",
                        vendor="https://hobbyking.com/en_us/turnigy-aerodrive-sk3-6374-192kv-brushless-outrunner-motor.html",
                        cost=85.85                  # $/unit
                       )

# Propeller
prop8x4_shape = ShapeCyl( # Describes the prop geometry as a disk
                        2.54*8/2 / 100,             # (m) radius
                        2.54*0.3/2 / 100,           # (m) height
                        "m"                         # Length units
                       )
prop8x4_obj = object_from_mass(                     # Calc density from mass&shape
                        prop8x4_shape,              # Shape
                        0.25*28.3495 / 1000;        # (kg) mass
                        massunits="kg"              # Mass units
                       )
prop8x4 = Component(                                # Component definition
                        "Propeller",                # Name
                        prop8x4_obj;                # Object
                        id="prop8x4",               # Identifier
                        description="APC 8x4.7 slow-flyer ropeller",
                        comments="",
                        vendor="https://www.apcprop.com/product/8x4-7sf/",
                        cost=2.45                   # $/unit
                       )


# Component database
componentdatabase = System(                          # System definition
                        "Component database",        # Name
                        [motor1000kv, motor3040kv, motor192kv, prop8x4]; # Subcomponents
                        description="Database of technology available"
              )

,Name,ID,Subcomponents,Description,Comments,Vendor,$/unit,Units,Total cost ($)
1,Component database,-1,"Motor 1000kv, Motor 3040kv, Motor 192kv, Propeller",Database of technology available,,,134.56,1,134.56
2,Motor 1000kv,motor1000kv,,Turnigy D2830-11 1000kv Brushless Motor,"1000kv, requires 30A ESC, suggested prop 8x4 (4S) ~ 10x7 (2S)",https://hobbyking.com/en_us/d2830-11-1000kv-brushless-motor.html,11,1,11
3,Motor 3040kv,motor3040kv,,Turnigy TrackStar 13.5T Sensored Brushless Motor 3040KV,"3040kv, max 36A ESC,",https://hobbyking.com/en_us/turnigy-trackstar-13-5t-sensored-brushless-motor-3040kv-roar-approved.html,35.26,1,35.26
4,Motor 192kv,motor192kv,,Turnigy Aerodrive SK3 - 6374-192KV Brushless Outrunner Motor,"192kv, max 80A ESC,",https://hobbyking.com/en_us/turnigy-aerodrive-sk3-6374-192kv-brushless-outrunner-motor.html,85.85,1,85.85
5,Propeller,prop8x4,,APC 8x4.7 slow-flyer ropeller,,https://www.apcprop.com/product/8x4-7sf/,2.45,1,2.45


## Shape Definitions

In [11]:
file_name = "trainer"     # Output file name

# ----------------- GEOMETRY DESCRIPTION -------------------------------------------
semispan = 1.0                       # (m) semi-span length

chords = [0.00 0.25;                # (semi-span position, chord c/semib)
          0.25 0.20;
          1.00 0.10]

twists = [0.0 5;                    # (semi-span position, twist (deg))
          1.0 0]

x_pos = [0.00 0;                    # (semi-span position, LE x-position x/semib)
         0.25 1/40;
         1.00 1/8;]

z_pos = [0.00 0;                    # (semi-span position, LE x-position x/semib)
         0.25 1/100;
         1.00 1/50]


airfoil_files = [(0.0, "naca6412.dat"), # (semi-span position, airfoil file)
                 (1.0, "naca6412.dat")]

# ----------------- MESHING PARAMETERS ---------------------------------------------
urfl_NDIVS = [(0.25, 10,   10.0, false),       # Cells on upper side of airfoils
              (0.50,  7,    1.0, true),
              (0.25,  8, 1/10.0, false)]                    
lrfl_NDIVS = urfl_NDIVS             # Cells on lower side of airfoils

####################################################################################
# Center section
####################################################################################
b_center = 0.25

# ----------------- MESHING PARAMETERS ---------------------------------------------
b_NDIVS = [(1.0, 20, 20.0, true)]   # Span cell sections


# ----------------- LOFTING PARAMETERS ---------------------------------------------
b_low = -b_center                   # Lower bound of span lofting 
b_up = b_center                     # Upper bound of span lofting
symmetric = true                    # Lofting symmetric about b=0
spl_k = 1                           # Spline order of distributions along span
spl_s = 0.0000001                   # Spline smoothing of distribution along span
verify_spline = false               # Plots the splined distributions
verify_rflspline = true             # Plots the splined airfoil cross sections
rflspl_s = 0.00000001               # Spline smoothing of airfoil cross sections.

# ----------------- GENERATE WING --------------------------------------------------
wing_orggrid = gt.generate_loft(airfoil_files, airfoil_path, urfl_NDIVS, lrfl_NDIVS, 
                                        semispan, b_low, b_up, b_NDIVS, 
                                        chords, twists, x_pos, z_pos; 
                                        symmetric=symmetric, 
                                        spl_k=spl_k, spl_s=spl_s,
                                        verify_spline=verify_spline,
                                        verify_rflspline=false, 
                                        rflspl_s=rflspl_s
                                    )

dimsplit = 1
wing_grid_center = gt.GridTriangleSurface(wing_orggrid, dimsplit)

# Save vtk and call paraview
gt.save(wing_grid_center, file_name*"_C"; path=save_path)


####################################################################################
# Left section
####################################################################################
# ----------------- MESHING PARAMETERS ---------------------------------------------
b_NDIVS = [(1.0, 20, 20.0, true)]   # Span cell sections


# ----------------- LOFTING PARAMETERS ---------------------------------------------
b_low = -1                          # Lower bound of span lofting 
b_up = -b_center                    # Upper bound of span lofting
symmetric = true                    # Lofting symmetric about b=0
spl_k = 1                           # Spline order of distributions along span
spl_s = 0.0000001                   # Spline smoothing of distribution along span
verify_spline = false               # Plots the splined distributions
verify_rflspline = true             # Plots the splined airfoil cross sections
rflspl_s = 0.00000001               # Spline smoothing of airfoil cross sections.

# ----------------- GENERATE WING --------------------------------------------------
wing_orggrid = gt.generate_loft(airfoil_files, airfoil_path, urfl_NDIVS, lrfl_NDIVS, 
                                        semispan, b_low, b_up, b_NDIVS, 
                                        chords, twists, x_pos, z_pos; 
                                        symmetric=symmetric, 
                                        spl_k=spl_k, spl_s=spl_s,
                                        verify_spline=verify_spline,
                                        verify_rflspline=false, 
                                        rflspl_s=rflspl_s
                                    )

dimsplit = 1
wing_grid_left = gt.GridTriangleSurface(wing_orggrid, dimsplit)

# Save vtk and call paraview
gt.save(wing_grid_left, file_name*"_L"; path=save_path)


####################################################################################
# Right section
####################################################################################
# ----------------- MESHING PARAMETERS ---------------------------------------------
b_NDIVS = [(1.0, 20, 20.0, true)]   # Span cell sections


# ----------------- LOFTING PARAMETERS ---------------------------------------------
b_low = b_center                    # Lower bound of span lofting 
b_up = 1                            # Upper bound of span lofting
symmetric = true                    # Lofting symmetric about b=0
spl_k = 1                           # Spline order of distributions along span
spl_s = 0.0000001                   # Spline smoothing of distribution along span
verify_spline = false               # Plots the splined distributions
verify_rflspline = true             # Plots the splined airfoil cross sections
rflspl_s = 0.00000001               # Spline smoothing of airfoil cross sections.

# ----------------- GENERATE WING --------------------------------------------------
wing_orggrid = gt.generate_loft(airfoil_files, airfoil_path, urfl_NDIVS, lrfl_NDIVS, 
                                        semispan, b_low, b_up, b_NDIVS, 
                                        chords, twists, x_pos, z_pos; 
                                        symmetric=symmetric, 
                                        spl_k=spl_k, spl_s=spl_s,
                                        verify_spline=verify_spline,
                                        verify_rflspline=false, 
                                        rflspl_s=rflspl_s
                                    )

dimsplit = 1
wing_grid_right = gt.GridTriangleSurface(wing_orggrid, dimsplit)


####################################################################################
# Horizontal tail
####################################################################################
tail_f = 5
tail_semispan = b_center*semispan*tail_f
# ----------------- MESHING PARAMETERS ---------------------------------------------
b_NDIVS = [(1.0, 20, 20.0, true)]   # Span cell sections


# ----------------- LOFTING PARAMETERS ---------------------------------------------
b_low = -1/tail_f                      # Lower bound of span lofting 
b_up = 1/tail_f                        # Upper bound of span lofting
symmetric = true                    # Lofting symmetric about b=0
spl_k = 1                           # Spline order of distributions along span
spl_s = 0.0000001                   # Spline smoothing of distribution along span
verify_spline = false               # Plots the splined distributions
verify_rflspline = true             # Plots the splined airfoil cross sections
rflspl_s = 0.00000001               # Spline smoothing of airfoil cross sections.

# ----------------- GENERATE WING --------------------------------------------------
wing_orggrid = gt.generate_loft(airfoil_files, airfoil_path, urfl_NDIVS, lrfl_NDIVS, 
                                        tail_semispan, b_low, b_up, b_NDIVS, 
                                        chords, twists, x_pos, z_pos; 
                                        symmetric=symmetric, 
                                        spl_k=spl_k, spl_s=spl_s,
                                        verify_spline=verify_spline,
                                        verify_rflspline=false, 
                                        rflspl_s=rflspl_s
                                    )

dimsplit = 1
tail_grid = gt.GridTriangleSurface(wing_orggrid, dimsplit)

# Save vtk and call paraview
gt.save(tail_grid, file_name*"_T"; path=save_path)

str = save_path*"/"*file_name*"_L.vtk;"*file_name*"_C.vtk;"*file_name*"_R.vtk;"*file_name*"_T.vtk;"
run(`paraview --data=$str`)

QApplication: invalid style override passed, ignoring it.
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] uuid4(::MersenneTwister) at ./deprecated.jl:57
 [3] msg_header at /home/edo/.julia/v0.6/IJulia/src/msg.jl:18 [inlined]
 [4] msg_pub(::IJulia.Msg, ::String, ::Dict{String,String}, ::Dict{String,Any}) at /home/edo/.julia/v0.6/IJulia/src/msg.jl:30 (repeats 2 times)
 [5] send_stream(::String) at /home/edo/.julia/v0.6/IJulia/src/stdio.jl:172
 [6] send_stdio(::String) at /home/edo/.julia/v0.6/IJulia/src/stdio.jl:130
 [7] (::Base.##302#303{IJulia.#send_stderr,Timer})() at ./event.jl:436
while loading In[11], in expression starting on line 172


## Component Definitions

In [12]:
# Wing center section
wingC_shape = adt.ShapeSurfGrid(                    # Arbitrary surface grid shape
                        wing_grid_center,           # Triangular surface grid
                        "m"                         # Length units
               )
wingC_obj = adt.ObjectVol(                          # Volumetric object
                        wingC_shape,                # Shape
                        24.8,                       # (kg/m^3) HD EPS foam density
                        "kg/m^3"                    # Density units
               )
wingC = adt.Component(                              # Component definition
                        "Wing",                     # Name
                        wingC_obj;                  # Object
                        id="wingHDeps",             # Identifier
                        cost=3.0                    # $/unit
                       )
# Wing left section
wingL_shape = adt.ShapeSurfGrid(                    # Arbitrary surface grid shape
                        wing_grid_left,             # Triangular surface grid
                        "m"                         # Length units
               )
wingL_obj = adt.ObjectVol(                          # Volumetric object
                        wingL_shape,                # Shape
                        24.8,                       # (kg/m^3) HD EPS foam density
                        "kg/m^3"                    # Density units
               )
wingL = adt.Component(                              # Component definition
                        "Wing",                     # Name
                        wingL_obj;                  # Object
                        id="wingHDeps",             # Identifier
                        cost=3.0                    # $/unit
                       )
# Wing right section
wingR_shape = adt.ShapeSurfGrid(                    # Arbitrary surface grid shape
                        wing_grid_right,            # Triangular surface grid
                        "m"                         # Length units
               )
wingR_obj = adt.ObjectVol(                          # Volumetric object
                        wingR_shape,                # Shape
                        24.8,                       # (kg/m^3) HD EPS foam density
                        "kg/m^3"                    # Density units
               )
wingR = adt.Component(                              # Component definition
                        "Wing",                     # Name
                        wingR_obj;                  # Object
                        id="wingHDeps",             # Identifier
                        cost=3.0                    # $/unit
                       )
# Tail
tail_shape = adt.ShapeSurfGrid(                    # Arbitrary surface grid shape
                        tail_grid,            # Triangular surface grid
                        "m"                         # Length units
               )
tail_obj = adt.ObjectVol(                          # Volumetric object
                        tail_shape,                # Shape
                        24.8,                       # (kg/m^3) HD EPS foam density
                        "kg/m^3"                    # Density units
               )
tail_wing = adt.Component(                              # Component definition
                        "Tail",                     # Name
                        tail_obj;                  # Object
                        id="tailHDeps",             # Identifier
                        cost=3.0                    # $/unit
                       )

# Boom
boom_length = 1.25*semispan
boom_shape = adt.ShapeCyl(                       # Cylindrical shape
                        0.01*semispan,                # (m) radius
                        boom_length,                  # (m) height
                        "m"                         # Length units
                       )
boom_obj = adt.ObjectVol(                          # Volumetric object
                        boom_shape,                # Shape
                        24.8,                       # (kg/m^3) HD EPS foam density
                        "kg/m^3"                    # Density units
               )
boom = adt.Component(                              # Component definition
                        "Boom",                     # Name
                        boom_obj;                  # Object
                        id="boomHDeps",             # Identifier
                        cost=3.0                    # $/unit
                       )

,Name,ID,Subcomponents,Description,Comments,Vendor,$/unit,Units,Total cost ($)
1,Boom,boomHDeps,,,,,3.0,1,3.0


## System Definitions

In [13]:
tilted = !true

# SYSTEMS

# Rotor pod
rotorpod = adt.System(
                        "Rotor pod",                # Name
                        [motor1000kv, prop8x4];     # Subcomponents
                        id="rotpod",                # Identifier
    
                        # Orientation of motor and prop: centerline (z-axis) 
                        # aligned with x-axis
                        subOaxis = [[0 1 0; 0 0 1; 1 0 0], [0 1 0; 0 0 1; 1 0 0]],
    
                        # Position of motor and prop: Rotor flush with motor face
                        subO=[zeros(3), [-prop8x4.subcomponents.shape.h, 0, 0]]
               )

# Left wing system
wingLsys = adt.System(
                        "WingLsys",                # Name
                        [wingL, rotorpod, rotorpod];     # Subcomponents
    
                        # Orientation of components: places the wing at 
                        # 5deg angle of attack relative to rotor axis
                        subOaxis = [adt.rotation_matrix(0, 5.0, 0), eye(3), eye(3)],
    
                        # Position of components
                        subO = [zeros(3), 
                                [0, -(b_center+0.15)*semispan, 0], 
                                [0.075, 0.95*semispan, 0]]
               )
        
# Right wing system
wingRsys = adt.System(
                        "WingRsys",                # Name
                        [wingR, rotorpod, rotorpod];     # Subcomponents
    
                        # Orientation of components: places the wing at 
                        # 5deg angle of attack relative to rotor axis
                        subOaxis = [adt.rotation_matrix(0, 5.0, 0), eye(3), eye(3)],
    
                        # Position of components
                        subO = [zeros(3), 
                                [0, (b_center+0.15)*semispan, 0], 
                                [0.075, -0.95*semispan, 0]]
               )

# Tail
tail = adt.System(
                        "TwinBoom",      # Name
                        [boom, boom, tail_wing];     # Subcomponents
                        id="twinboom",     # Identifier
    
                        subOaxis = [adt.rotation_matrix(0, 90, 90), 
                                    adt.rotation_matrix(0, 90, 90), 
                                    eye(3)],
    
                        # This is garbage
                        subO = [[0.175, 0.95*b_center*semispan, 0], 
                                [0.175, -0.95*b_center*semispan, 0], 
                                [0.175+boom_length-0.1, 0, 0]]
               )

# Aircraft system
aircraft = adt.System(
                        "Tilt-wing Aircraft",      # Name
                        [wingLsys, wingC, wingRsys, tail];     # Subcomponents
                        id="tiltwingaircraft",     # Identifier
    
                        # Orientation of wing sections: outboard sections
                        # tilted vertically for VTOL, and center section
                        # at 5deg angle of attack
                        subOaxis = [tilted ? adt.rotation_matrix(0, 90, 0) : eye(3), 
                                    adt.rotation_matrix(0, 5.0, 0), 
                                    tilted ? adt.rotation_matrix(0, 90, 0) : eye(3),
                                    eye(3)],
    
                        # This is garbage
                        subO = [tilted ? [0.08, 0, 0.1125] : zeros(3), 
                                zeros(3), 
                                tilted ? [0.08, 0, 0.1125] : zeros(3), 
                                zeros(3)]
               )

str = adt.save_shape(aircraft; path=save_path)

run(`paraview --data=$(save_path)/$(str)`)

QApplication: invalid style override passed, ignoring it.
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] uuid4(::MersenneTwister) at ./deprecated.jl:57
 [3] msg_header at /home/edo/.julia/v0.6/IJulia/src/msg.jl:18 [inlined]
 [4] msg_pub(::IJulia.Msg, ::String, ::Dict{String,String}, ::Dict{String,Any}) at /home/edo/.julia/v0.6/IJulia/src/msg.jl:30 (repeats 2 times)
 [5] send_stream(::String) at /home/edo/.julia/v0.6/IJulia/src/stdio.jl:172
 [6] send_stdio(::String) at /home/edo/.julia/v0.6/IJulia/src/stdio.jl:130
 [7] (::Base.##302#303{IJulia.#send_stderr,Timer})() at ./event.jl:436
while loading In[13], in expression starting on line 88


In [14]:
display(aircraft)

,Name,ID,Subcomponents,Description,Comments,Vendor,$/unit,Units,Total cost ($)
1,Tilt-wing Aircraft,tiltwingaircraft,"WingLsys, Wing, WingRsys, TwinBoom",,,,71.8,1,71.8
2,WingLsys,-1,"Wing, Rotor pod, Rotor pod",,,,29.9,1,29.9
3,Wing,wingHDeps,,,,,3.0,1,3.0
4,Rotor pod,rotpod,"Motor 1000kv, Propeller",,,,13.45,4,53.8
5,Motor 1000kv,motor1000kv,,Turnigy D2830-11 1000kv Brushless Motor,"1000kv, requires 30A ESC, suggested prop 8x4 (4S) ~ 10x7 (2S)",https://hobbyking.com/en_us/d2830-11-1000kv-brushless-motor.html,11,4,44
6,Propeller,prop8x4,,APC 8x4.7 slow-flyer ropeller,,https://www.apcprop.com/product/8x4-7sf/,2.45,4,9.8
7,Wing,wingHDeps,,,,,3.0,1,3.0
8,WingRsys,-1,"Wing, Rotor pod, Rotor pod",,,,29.9,1,29.9
9,Wing,wingHDeps,,,,,3.0,1,3.0
10,TwinBoom,twinboom,"Boom, Boom, Tail",,,,9.0,1,9.0


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] uuid4(::MersenneTwister) at ./deprecated.jl:57
 [3] msg_header at /home/edo/.julia/v0.6/IJulia/src/msg.jl:18 [inlined]
 [4] msg_pub(::IJulia.Msg, ::String, ::Dict{String,Any}, ::Dict{String,Any}) at /home/edo/.julia/v0.6/IJulia/src/msg.jl:30 (repeats 2 times)
 [5] display(::IJulia.InlineDisplay, ::AircraftDesignTools.System{AircraftDesignTools.AbstractComponent,Float64,Float64}) at /home/edo/.julia/v0.6/IJulia/src/inline.jl:88
 [6] display(::AircraftDesignTools.System{AircraftDesignTools.AbstractComponent,Float64,Float64}) at ./multimedia.jl:218
 [7] include_string(::String, ::String) at ./loading.jl:522
 [8] softscope_include_string(::Module, ::String, ::String) at /home/edo/.julia/v0.6/SoftGlobalScope/src/SoftGlobalScope.jl:66
 [9] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/edo/.julia/v0.6/IJulia/src/execute_request.jl:193
 [10] (::Compat.#inner#6{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJ